In [1]:
# Import necessary libraries
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import math
import string
import re
from sklearn.preprocessing import LabelEncoder
import os
import pickle
import sys
from IPython.display import display, HTML
import urllib
import gzip
import nltk
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk.stem
from sklearn.metrics import classification_report,confusion_matrix
from nltk import word_tokenize
from nltk.corpus import wordnet, stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn import preprocessing
from sklearn.neural_network import MLPClassifier
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import Normalizer
from sklearn.model_selection import  train_test_split
from scipy.sparse import hstack
from nltk import tokenize
nltk.download('punkt')
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
import pip

[nltk_data] Downloading package punkt to C:\Users\Kumar
[nltk_data]     Ayush\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Kumar
[nltk_data]     Ayush\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Kumar
[nltk_data]     Ayush\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
def cleanString(review):
    # Setup the stopwords from the English dictionary
    #This can be used to clean Entire Dataframe of Documents.
    stopWords = set(stopwords.words('english'))
    #Create an object for word lemmatizer
    lemmatizer = WordNetLemmatizer()
    returnString = ""
    #Tokenize the words from string to a list
    sentence_token = tokenize.word_tokenize(review)
    idx_list = []
    for j in range(len(sentence_token)):
        #For each sentence after tokenization, we take in the word for processing
        single_sentence = tokenize.word_tokenize(sentence_token[j])
        #We apply the word lemmetizer to lemmatize the word where we remove verb , adjactive inorder to provide the required word in the sentence.
        single_sentence=[lemmatizer.lemmatize(t) for t in single_sentence]
        # Filter stop words , make the all words in lower character and enumerate the sentence to remove special characters.
        single_sentence=[word for word in single_sentence if word.lower() not in stopWords]
        sentences_filtered = [(idx,lemmatizer.lemmatize(w.lower())) for idx,w in enumerate(single_sentence) 
                              if w.lower() not in stopWords and w.isalnum()]
        idx_list.append([x[0] for x in sentences_filtered])
        word_list = [x[1] for x in sentences_filtered]
        returnString = returnString + ' '.join(word_list) + ' '
    
    return returnString





def train_validation(model,train,y_train,validation,y_test):
 #APPLY CHARACTER VECTORIZER TRANSFORM ON VALIDATION DATA, 20000 dimensions of characters are created
  #APPLY WORD VECTORIZER TRANSFORM ON VALIDATION DATA, 3000 dimensions of words are created
     #LOWER THE DIMENSION USING SVD IN PRINCIPAL COMPONENT ANALYSIS TO VERY FEW 500 DIMENSIONS
        #Normalize the incoming test stack and train data
         #FIT THE MODEL and SAVE THE FITTED MODEL
            # APPLY UNSUPERVISED METHOD PAIRWAISE COSINE DISTANCE BETWEEN THE INCOMING TEXT and DOCUMENT and FIND THE CLOSEST MATCH 
                #Unsupervised method ensures the words are matched with context of the document and closest match is choosen for prediction
                
    X_train_char=char_vectorizer.transform(train)
    X_train_word=word_vectorizer.transform(train)
    train_stack = hstack([X_train_char,X_train_word])
    
    X_test_char = char_vectorizer.transform(validation)
    X_test_word = word_vectorizer.transform(validation)
    test_stack = hstack([X_test_char,X_test_word])
    
    svd_model=pickle.load( open( file1+"_SVD_preprocessing.sav", "rb" ) )
    train_stack=svd_model.transform(train_stack)
    test_stack=svd_model.transform(test_stack)
    
    train_features2 = Normalizer(copy=False).fit_transform(train_stack)
    test_stack = Normalizer(copy=False).transform(test_stack)
    
    
    fitted_model=model.fit(train_features2, y_train)
    filename = file1+"MLP"+'_model.sav'
    pickle.dump(fitted_model, open(filename, 'wb'))
    
    

    
    
    best_thread = pairwise_distances_argmin(
            X=test_stack,
            Y=train_features2,
            metric='cosine'
        )
    
    
  
    
    y_pred_class1 = model.predict(train_features2[best_thread])
    y_pred_class2 = model.predict(test_stack)
    
   
   
   
    print("Training Accuracy")
    print(model.score(train_features2,y_train))
    print("Testing Accuracy1")
    print(model.score(test_stack,y_test))
    print("Testing Accuracy2")
    print(model.score(train_features1[best_thread],y_test))
    return(y_pred_class1)





In [3]:
file1 = "Combined_Corpus"
stats=pd.read_csv(file1+".csv")
labelEncoder = LabelEncoder()
response = dict(zip(stats.Topic_Categories, stats.Response))


stats["Topic_Categories"] = labelEncoder.fit_transform(stats["Topic_Categories"])

import pickle

with open(file1+'_labelEncoder.sav', 'wb') as file:
    pickle.dump(labelEncoder, file, pickle.HIGHEST_PROTOCOL)
    
    
with open(file1+'_ResponseDictionary.sav', 'wb') as file:
    pickle.dump(response, file, pickle.HIGHEST_PROTOCOL)
    
    
stats = stats.rename(columns={'Topic_Categories': 'Category', 'Question': 'Text'})
data_df=stats.sample(frac=1).reset_index(drop=True)
data_df["Text"]=data_df.Text.apply(cleanString)
y=data_df["Category"]
X=data_df["Text"]

with open(file1+'_Data_df.sav', 'wb') as file:
    pickle.dump(data_df, file, pickle.HIGHEST_PROTOCOL)

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.01, random_state=42)


def build_tokenizer(doc):
    token_pattern=r"(?u)\b\w+"
    token_pattern = re.compile(token_pattern)
    return token_pattern.findall(doc)


english_stemmer = nltk.stem.SnowballStemmer('english')
class StemmedTfidfVectorizer(TfidfVectorizer):
    def build_analyzer(self):
        analyzer = super(TfidfVectorizer,self).build_analyzer()
        return lambda doc:(english_stemmer.stem(word) for word in analyzer(doc))
    
posts_root1=[]

for post in X_train:
    a=" ".join([english_stemmer.stem(word) for word in build_tokenizer(post)])
    posts_root1.append(a)



word_vectorizer = StemmedTfidfVectorizer(
    sublinear_tf=True,
    stop_words = 'english',
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    ngram_range=(1,7),
    norm='l2',
    dtype=np.float32,
    max_features=3000,
    
)
# Character Stemmer
char_vectorizer = StemmedTfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='char',
    ngram_range=(1, 10),
    dtype=np.float32,
    norm='l2',
    max_features=20000,
   
)



word_vectorizer.fit(posts_root1)
char_vectorizer.fit(posts_root1)


train_word_features = word_vectorizer.transform(posts_root1)
train_char_features = char_vectorizer.transform(posts_root1)



train_features = hstack([
    train_char_features,
    train_word_features])
tsvd=TruncatedSVD(1000)

train_features1 = tsvd.fit_transform(train_features)
filename = file1+'train_features1_preprocessing.sav'
pickle.dump(train_features1, open(filename, 'wb'))
filename = file1+'_word_preprocessing.sav'
pickle.dump(word_vectorizer, open(filename, 'wb'))
filename = file1+'_char_preprocessing.sav'
pickle.dump(char_vectorizer, open(filename, 'wb'))
filename = file1+'_SVD_preprocessing.sav'
pickle.dump(tsvd, open(filename, 'wb'))


print("char_feature_name:%s" % len(char_vectorizer.get_feature_names()))
print("word_feature_name:%s" % len(word_vectorizer.get_feature_names()))
print("Explained Variance:%s" % tsvd.explained_variance_ratio_.sum())

char_feature_name:20000
word_feature_name:2587
Explained Variance:1.0


In [4]:
stats.columns=["Topic_Code","Topic_Categories","Question","Response"]
data_df=stats.sample(frac=1).reset_index(drop=True)
data_df["Question"]=data_df.Question.apply(cleanString)


y=data_df["Topic_Categories"]
X=data_df["Question"]
X_train1, X_test1, y_train1, y_test1 = train_test_split(X,y,test_size=0.3, random_state=42)
mlp=MLPClassifier(activation='relu', alpha=0.001, batch_size='auto', beta_1=0.75,
              beta_2=0.8, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(300, 300), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=200,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=200, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)
from sklearn.metrics.pairwise import pairwise_distances_argmin



train_validation(mlp,X_train,y_train,X_test1,y_test1)

Training Accuracy
0.9924698795180723
Testing Accuracy1
0.9900990099009901
Testing Accuracy2
0.9504950495049505


array([72, 72, 36, 55,  2, 91, 72, 18, 42,  0, 69, 64, 82, 73, 80,  4,  4,
       47, 80, 47,  9, 36, 70, 58, 47, 90, 75, 54, 74, 73, 81, 65, 55, 64,
       54, 49, 80, 54, 53, 80,  3, 50, 56, 64, 40, 43, 52, 60, 40, 55, 66,
       40, 69, 30, 82, 83, 39, 92, 65, 27, 67, 40, 94,  1, 69, 62, 30, 79,
       60, 80, 73, 85, 47, 30,  3, 57,  8, 81, 88, 60, 63, 93, 67, 69, 55,
       70, 30, 49, 51, 62, 93, 42, 65, 58, 61, 42, 43, 93, 56, 58, 85, 40,
       54, 36, 55, 55,  8, 55, 95, 49, 75, 27, 70, 74, 56, 81, 75, 36, 65,
       93, 22, 81, 90, 20, 93, 43, 91, 56, 73, 89, 56, 74, 92, 54, 93, 79,
       72, 57, 89, 12, 65, 81,  2, 41, 87, 28,  4, 69, 30, 35, 54, 86, 65,
       45, 59, 65, 73, 56, 94, 51, 54, 61, 41, 80, 58, 49, 56, 55, 36, 61,
       60, 80, 56, 60, 95, 72, 60, 34, 95, 64, 49, 40, 55, 49, 51, 73, 77,
       53, 95, 89, 90, 49,  4, 61, 40, 89, 55, 38, 29, 60, 74, 94])